# Libraries Import

In [111]:
from sklearn.model_selection import KFold
from IPython.display import display
import random
import numpy as np
import pandas as pd
import re
import os

# Lectura de datos

In [112]:
df_Processed_Data = pd.read_csv('./data/dataClean/df_Processed_Data.csv')

In [113]:
df_Processed_Data.head(5)

,ID,Title,Keywords,FirstLevel,Processed_Title,Processed_Keyword,Processed_T_K
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",['H'],process-oriented structures,cooperative knowledge generation,process-oriented structures cooperative knowle...
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...",['H' 'J'],small groups learning synchronously online,computer-supported cooperative learning,small groups learning synchronously online com...
2,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...",['A' 'D' 'H' 'J' 'K'],knowledge sharing,micro-didactical learning arrangement,knowledge sharing micro-didactical learning ar...
3,5,Modelling and Implementing Pre-built Informati...,"modelling method,introduction method,context-a...",['H' 'I' 'J'],implementing pre-built information spaces,modelling method,implementing pre-built information spaces mode...
4,6,Tube Map Visualization: Evaluation of a Novel ...,"knowledge visualization,information visualiza...",['H'],tube map visualization,knowledge visualization,tube map visualization knowledge visualization


In [187]:
isHadNull = df_Processed_Data.isnull().sum()

print(isHadNull)

ID                   0
Title                0
Keywords             0
FirstLevel           0
Processed_Title      0
Processed_Keyword    0
Processed_T_K        0
dtype: int64


In [115]:
df_clean = pd.read_csv('./data/dataClean/df_clean.csv')


# Obtener el mejor thresold

In [116]:
def getThresold(population,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train,exp_list,threshold):
  # aca la funcion fittness sera trabajoso basicamente es el algoritmo fuzzy classifier algorithm mas la parte de evaluacion multietiqueta

  fitnessByChromosome =[]
  thresholdByChromosome =[]

  cnt_p = 1

  for chromosome in population:

    bestThresold = 0
    bestAcurracy = 0
    print("\n ------------------------------------ POBLACION: ", cnt_p," ------------------------------------ \n")
    thres = threshold
    cont = 0
    while(thres < 1.0):

      pred_list_kt = []
      pred_list_k = []
      pred_list_t = []
      print("Con threshold: ", thres);

      cnt_d = 1
      for tf in dataFBRM_train: #TF by doc
        # print("\nDOCUMENTO TF",cnt_d,"\n")
        pred_keywords_title, pred_keywords, pred_title = FuzzyClassifierAlgorithm(chromosome,df_featuresMap,nColumns,tf,thres)# get predicted class by Doc
        pred_list_kt.append(pred_keywords_title)
        pred_list_k.append(pred_keywords)
        pred_list_t.append(pred_title)
        cnt_d += 1


      labels = sorted(set([label for sublist in exp_list for label in sublist]))

      y_true_bin = np.zeros((len(exp_list), len(labels)))
      y_pred_bin = np.zeros((len(pred_list_kt), len(labels)))

      for i, label_list in enumerate(exp_list):
          for label in label_list:
              y_true_bin[i, labels.index(label)] = 1

      for i, label_list in enumerate(pred_list_kt):
          for label in label_list:
              y_pred_bin[i, labels.index(label)] = 1

      # Calcular la precisión (accuracy) para cada etiqueta y promediar
#       macro_accuracy = 0
#       for i in range(len(labels)):
#           true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
#           true_negatives = np.sum(np.logical_and(1 - y_true_bin[:, i], 1 - y_pred_bin[:, i]))
#           total_samples = true_positives + true_negatives + np.sum(np.logical_xor(y_true_bin[:, i], y_pred_bin[:, i]))
#           label_accuracy = (true_positives + true_negatives) / total_samples
#           macro_accuracy += label_accuracy

#       macro_accuracy /= len(labels)
      macro_f1 = 0
      for i in range(len(labels)):
          true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
          false_positives = np.sum(np.logical_and(1 - y_true_bin[:, i], y_pred_bin[:, i]))
          false_negatives = np.sum(np.logical_and(y_true_bin[:, i], 1 - y_pred_bin[:, i]))

          precision = true_positives / (true_positives + false_positives + 1e-9)
          recall = true_positives / (true_positives + false_negatives + 1e-9)

          f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
          macro_f1 += f1_score

      macro_f1 /= len(labels)

      if(macro_f1 > bestAcurracy):

        bestThresold = thres
        bestAcurracy = macro_f1
        print("the best Thresold: ", bestThresold)
        print("the best Acurracy: ", bestAcurracy)
        cont = 0
      else:
        cont +=1

      if (cont > 5):
            break;

      thres+=0.01

    cnt_p +=1
    fitnessByChromosome.append(bestAcurracy)
    thresholdByChromosome.append(bestThresold)

  return fitnessByChromosome, thresholdByChromosome

#  Fuzzy Based Rules Merger (FBRM) algorithm

### Get term frequency per item (TF)

In [117]:
from nltk import FreqDist

In [118]:
def count_terms_frecuency(r):

  freq_dist = FreqDist(r)
  term_frequency = dict(freq_dist)

  return term_frequency

In [119]:
def algorithmFBRM(category,data_):
  k = [] #array of keywords of category c
  t = [] #array of title of category c
  r = [] # resultant array of k and t of category c

  for i in range(len(data_)):
    titles = data_.iloc[i]['Processed_Title'].split()
    keywords = data_.iloc[i]['Processed_Keyword'].split()
    t_k = data_.iloc[i]['Processed_T_K'].split()
    
    for text in titles:
      k.append(text)
    for text in keywords:
      t.append(text)
    for text in t_k:
      r.append(text)
  return k, t, r

# GENETIC ALGORITHM

### Number of parameters
> Cantidad de individuos: 10

> Cantidad de genes por individuos: ---

> Seleccion por torneo

> Probabilidad de Cruzamiento: 0.9

> Cruzamiento de BLX-alpha :0.5

> Probabilidad de mutacion: 0.5

> Mutacion de Intercambio

> Cantidad de iteraciones: 2000

## Population generation

In [120]:
def genPopulation(nInd,data_tf):
  datadf_chromosome = data_tf[data_tf['Metadata'] != 'tf_keywords_title']
  # print(datadf_chromosome)
  df_features = datadf_chromosome.iloc[:, 2:]
  # print(df_features.shape)
  population = []
  flattened_list = df_features.values.ravel().tolist()
  # print(flattened_list)

  for i in range(nInd):
    # population.append([int(round(random.uniform(gen - (gen * 0.1), gen + (gen * 0.1)))) for gen in flattened_list])
    # population.append([random.uniform(gen - (gen * 0.1), gen + (gen * 0.1)) for gen in flattened_list])
    # -1 y 1
    population.append([gen + ((random.random() * 2 - 1) * gen * 0.1) for gen in flattened_list])

  return np.array(population), df_features.shape[1]

## Function Fitness

In [194]:
# classify the doc to a or more categories 

# chromosome, list of genes (list)
# df_featuresMap, panda of features (dataframe)
# nColumns, quantity of gens (int)
# tf, term-frecuency by doc (object = {word1:1, word2:2 ...})
# threshold, pre-threshold  (float)

def FuzzyClassifierAlgorithm(chromosome, df_featuresMap, nColumns, tf, threshold):
    pred_keywords_title = []
    pred_keywords = []
    pred_title = []

    columns = list(df_featuresMap.iloc[:, 2:].columns)
    nRow = len(chromosome) // nColumns

    tf_keywords_title = df_featuresMap[df_featuresMap['Metadata'] == 'tf_keywords_title']

    w_keywords = np.zeros(len(tf_keywords_title))
    w_title = np.zeros(len(tf_keywords_title))
    w_keywords_title = np.zeros(len(tf_keywords_title))

    c_list = tf_keywords_title['Names'].tolist() # 13 COMPONENTS, it is the categories list

    tf_i = 0
    tf_j = 0

    for i in range(nRow):
      if (i % 2 == 0):
        # to keyword
          idx_arr = [[columns.index(key), value] for key,value in tf.items() if key in columns]
          wk_k = sum([chromosome[i*nColumns + idx[0]] * idx[1] for idx in idx_arr]) # sumatoria(v_tfTble * tfbyDoc)
          w_keywords[tf_i] = wk_k
          tf_i += 1
      else:
        # to title
          idx_arr = [[columns.index(key), value] for key,value in tf.items() if key in columns]
          wk_t = sum([chromosome[i*nColumns + idx[0]] * idx[1] for idx in idx_arr]) # sumatoria(v_tfTble * tfbyDoc)
          w_title[tf_j] = wk_t
          tf_j += 1

    tf_tk = 0
    # to keyword + title
    for i in range(nRow):
      if (i % 2 == 0):
          idx_arr = [[columns.index(key), value] for key,value in tf.items() if key in columns]
          wk_kt = sum([(chromosome[i*nColumns + idx[0]] + chromosome[(i+1)*nColumns + idx[0]]) * idx[1] for idx in idx_arr]) # sumatoria(v_tfTble * tfbyDoc)
          w_keywords_title[tf_tk] = wk_kt
          tf_tk += 1


    # print("KEYWORDS")
    # print(w_keywords)
    # print("TITLE")
    # print(w_title)
    # print("KEYWORD + TITLE")
    # print(w_keywords_title)

    weightSum = w_keywords.sum()
    uD = w_keywords / weightSum
    pred_keywords = [c_list[i] for i in range(len(uD)) if uD[i] >= threshold]

    weightSum = w_title.sum()
    uD = w_title / weightSum
    pred_title = [c_list[i] for i in range(len(uD)) if uD[i] >= threshold]

    weightSum = w_keywords_title.sum()
    uD = w_keywords_title / weightSum
    pred_keywords_title = [c_list[i] for i in range(len(uD)) if uD[i] >= threshold]

    return pred_keywords_title, pred_keywords, pred_title


In [122]:
# population, list of list (matriz)
# df_featuresMap, pandas table of features (pandas)
# nColumns, nro of columns of features (int)
# nInd, nro of chromosome (int)
# nGenes nro of columns of population (int)
# dataFBRM_train list of object (tf of each doc) (list)
# exp_list list of experated categories (list)
# threshold (float)
def getFittness(population,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train,exp_list,threshold):
  # aca la funcion fittness sera trabajoso basicamente es el algoritmo fuzzy classifier algorithm mas la parte de evaluacion multietiqueta


  fitnessByChromosome = []
  predictionList = []

  cnt_p = 1


  for chromosome in population:

    print("\n ------------------------------------ POBLACION: ", cnt_p," ------------------------------------ \n")

    pred_list_kt = []
    pred_list_k = []
    pred_list_t = []

    cnt_d = 1
    for tf in dataFBRM_train:
      # print("\nDOCUMENTO TF",cnt_d,"\n")
      pred_keywords_title, pred_keywords, pred_title = FuzzyClassifierAlgorithm(chromosome,df_featuresMap,nColumns,tf,threshold) # get predicted class by Doc
      pred_list_kt.append(pred_keywords_title)
      pred_list_k.append(pred_keywords)
      pred_list_t.append(pred_title)
      cnt_d += 1

    labels = sorted(set([label for sublist in exp_list for label in sublist]))

    y_true_bin = np.zeros((len(exp_list), len(labels)))
    y_pred_bin = np.zeros((len(pred_list_kt), len(labels)))

    for i, label_list in enumerate(exp_list):
        for label in label_list:
            y_true_bin[i, labels.index(label)] = 1

    for i, label_list in enumerate(pred_list_kt):
        for label in label_list:
            y_pred_bin[i, labels.index(label)] = 1

    # Calcular la precisión (accuracy) para cada etiqueta y promediar
#     macro_accuracy = 0
#     for i in range(len(labels)):
#      true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
#      true_negatives = np.sum(np.logical_and(1 - y_true_bin[:, i], 1 - y_pred_bin[:, i]))
#      total_samples = true_positives + true_negatives + np.sum(np.logical_xor(y_true_bin[:, i], y_pred_bin[:, i]))
#      label_accuracy = (true_positives + true_negatives) / total_samples
#      macro_accuracy += label_accuracy

#     macro_accuracy /= len(labels)
    # Calculate the precision, recall, and F1-score for each label and average them for macro average F1-score
    macro_f1 = 0
    for i in range(len(labels)):
       true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
       false_positives = np.sum(np.logical_and(1 - y_true_bin[:, i], y_pred_bin[:, i]))
       false_negatives = np.sum(np.logical_and(y_true_bin[:, i], 1 - y_pred_bin[:, i]))

       precision = true_positives / (true_positives + false_positives + 1e-9)
       recall = true_positives / (true_positives + false_negatives + 1e-9)

       f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
       macro_f1 += f1_score

    macro_f1 /= len(labels)
    cnt_p +=1

    fitnessByChromosome.append(macro_f1)
    predictionList.append(pred_list_kt)

  return fitnessByChromosome, predictionList

In [195]:
def getFittnessToAccuracy(population,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train,exp_list,threshold):
  # aca la funcion fittness sera trabajoso basicamente es el algoritmo fuzzy classifier algorithm mas la parte de evaluacion multietiqueta


  fitnessByChromosome = []
  predictionList = []

  cnt_p = 1


  for chromosome in population:

    print("\n ------------------------------------ POBLACION: ", cnt_p," ------------------------------------ \n")

    pred_list_kt = []
    pred_list_k = []
    pred_list_t = []

    cnt_d = 1
    for tf in dataFBRM_train:
      # print("\nDOCUMENTO TF",cnt_d,"\n")
      pred_keywords_title, pred_keywords, pred_title = FuzzyClassifierAlgorithm(chromosome,df_featuresMap,nColumns,tf,threshold) # get predicted class by Doc
      pred_list_kt.append(pred_keywords_title)
      pred_list_k.append(pred_keywords)
      pred_list_t.append(pred_title)
      cnt_d += 1

    labels = sorted(set([label for sublist in exp_list for label in sublist]))

    y_true_bin = np.zeros((len(exp_list), len(labels)))
    y_pred_bin = np.zeros((len(pred_list_kt), len(labels)))

    for i, label_list in enumerate(exp_list):
        for label in label_list:
            y_true_bin[i, labels.index(label)] = 1

    for i, label_list in enumerate(pred_list_kt):
        for label in label_list:
            y_pred_bin[i, labels.index(label)] = 1

    # Calcular la precisión (accuracy) para cada etiqueta y promediar
    macro_accuracy = 0
    for i in range(len(labels)):
      true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
      true_negatives = np.sum(np.logical_and(1 - y_true_bin[:, i], 1 - y_pred_bin[:, i]))
      total_samples = true_positives + true_negatives + np.sum(np.logical_xor(y_true_bin[:, i], y_pred_bin[:, i]))
      label_accuracy = (true_positives + true_negatives) / total_samples
      macro_accuracy += label_accuracy

    macro_accuracy /= len(labels)
    # Calculate the precision, recall, and F1-score for each label and average them for macro average F1-score
#     macro_f1 = 0
#     for i in range(len(labels)):
#        true_positives = np.sum(np.logical_and(y_true_bin[:, i], y_pred_bin[:, i]))
#        false_positives = np.sum(np.logical_and(1 - y_true_bin[:, i], y_pred_bin[:, i]))
#        false_negatives = np.sum(np.logical_and(y_true_bin[:, i], 1 - y_pred_bin[:, i]))

#        precision = true_positives / (true_positives + false_positives + 1e-9)
#        recall = true_positives / (true_positives + false_negatives + 1e-9)

#        f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
#        macro_f1 += f1_score

#     macro_f1 /= len(labels)
    cnt_p +=1

    fitnessByChromosome.append(macro_accuracy)
    predictionList.append(pred_list_kt)

  return fitnessByChromosome, predictionList

## Seleccion por Torneo

In [124]:
def __maximize__(arr,x,y):
  idx = x if (arr[x] >= arr[y]) else y
  # print(arr[x] ,"\t- ", arr[y], "\t=> ",arr[idx])
  return idx

In [125]:
# by tournament
def selection(population,arrFittness,nInd):

  print("\nCreacion de Mating Pool\n")
  mating_pool = []

  for i in range(nInd):
    idxCouple1 = random.randint(0, nInd-1)
    idxCouple2 = random.randint(0, nInd-1)
    idxMax = __maximize__(arrFittness,idxCouple1,idxCouple2)
    mating_pool.append(population[idxMax])
    print(idxCouple1 ,"\t- ", idxCouple2, "\t=> ", idxMax, "\t=> ", population[idxMax])

    # print(idxCouple1)
    # print(idxCouple2)
    # print("menor: ",)
  return np.array(mating_pool)

## CROSSOVER

In [126]:
def blend_crossover_blx_alpha(father1,father2,pCrossover_BLX_Alpha,pCrossover):
    # offspring  = np.zeros_like(father1)
    c1 = 0
    c2 = 0
    nGenes = len(father1)
    betha = random.uniform((-1) * pCrossover_BLX_Alpha,1 + pCrossover_BLX_Alpha)

    child = []
    for i in range(nGenes):
      newGen = father1[i] + (betha*abs(father2[i] - father1[i]))
      child.append(newGen)
    return np.array(child)

In [127]:
# by blx-alpha
def crossover(mating_pool,pCrossover,nInd,cross,pCrossover_BLX_Alpha):

  child = []
  parent = []
  for i in range(nInd):
    idxCouple1 = random.randint(0, nInd-1)
    idxCouple2 = random.randint(0, nInd-1)

    print("\nSELECCION DE PADRES")
    couple_1 = mating_pool[idxCouple1]
    couple_2 = mating_pool[idxCouple2]
    print(idxCouple1, " - ", idxCouple2, " =>  ", couple_1, " - ", couple_2)

    if (random.random() <= pCrossover):
      offstring = cross(couple_1,couple_2,pCrossover_BLX_Alpha,pCrossover)
      child.append(offstring)
      print("\nCRUZAMIENTO")
      print(offstring)

    else:
      print("\nSin Cruzamiento")
      parent.append(couple_1)


  return np.array(child), np.array(parent)

In [128]:
## MUTATION of 10% with value float

## MUTATION of 10% with value float

In [129]:
def mutate(children,pMutation,nGenes):

  for i in range(len(children)):
    for j in range(len(children[i])):
      if (random.random() <= pMutation):

        # print("\nMUTACION")
        gen = children[i][j]
        # print("Posicion: ", j, " => ", children[i][j], "\t=> valor actual: ", gen)
        # children[i][j] = round(random.uniform(gen - (gen * 0.1), gen + (gen * 0.1)))
        children[i][j] = gen + ((random.random() * 2 - 1) * gen * 0.1)
        # population.append([gen + (random.random() * gen * 0.1) for gen in flattened_list])
        # print("Mutated: ", j, " => ", children[i][j], "\t=> valor mutado: ", children[i][j], "\n")
      # else :
        # print("\nSin Mutacion\n")
  return children

## Crossover and Mutate

In [130]:
def crossover_mutate(mating_pool,pCrossover,nInd,pMutation,nGenes,cross,pCrossover_BLX_Alpha):
  children, parents = crossover(mating_pool,pCrossover,nInd,cross,pCrossover_BLX_Alpha)

  print("\nMUTATION\n")

  if (len(children) == 0): # if there isn't children then return only parents
    return parents
  childrenMuta = mutate(children,pMutation,nGenes)

  if(len(parents) == 0): # if there isn't parents then return only children
    return childrenMuta
  offstrings = np.concatenate((childrenMuta, parents), axis=0)

  return offstrings

## Funcion del Algoritmo Genetico

In [131]:
def geneticAlgorithms(train_data = "",nInd = 16,df_featuresMap = "",dataFBRM_train_kt = "",pCrossover = 0.7,pMutation = 0.05,iteration = 100,cross = "", pCrossover_BLX_Alpha = 0.5,threshold = 0.1):

  #Seleccion de individuos nIndiM
  newGeneration, nColumns = genPopulation(nInd,df_featuresMap)
  
  ## numero de genes
  nGenes = newGeneration.shape[1]
  ### lista de categorias
  exp_list = train_data['FirstLevel'].tolist()


  print("OBTENER EL MEJOR THRESHOLD")
  ### obtener el mejor threshold
  candidateGeneration = []
  idxChromosome = random.randint(0, nInd-1)
  candidateGeneration.append(newGeneration[idxChromosome])

  fitnessByChromosome, thresholdByChromosome = getThresold(candidateGeneration,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train_kt,exp_list,threshold)
  print("mejores resultados: ",fitnessByChromosome, "\t-> mejor thresold",thresholdByChromosome)
  idx_max= fitnessByChromosome.index(max(fitnessByChromosome))
  bestThresold = thresholdByChromosome[idx_max]


  for i in range(iteration):
    print("\n-------ITERACION ",i+1," ----------\n")

    #Calcular Fittness
    f_, predictionList = getFittness(newGeneration,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train_kt,exp_list,bestThresold)

    print("\n NUEVA GENERACION ")
    print("Generación \t\t-> fitness")
    print(newGeneration, "\t\t->", f_)

    # Seleccion de Mating pool
    mating_pool = selection(newGeneration,f_,nInd)

    # Seleccion de padres
    # para cruzamiento
    # mutacion
    newGeneration = crossover_mutate(mating_pool,pCrossover,nInd,pMutation,nGenes,cross,pCrossover_BLX_Alpha)

  print("\n\nCondicion de Parada")

  f_, predictionList = getFittness(newGeneration,df_featuresMap,nColumns,nInd,nGenes,dataFBRM_train_kt,exp_list,bestThresold)

  print("\n\nFIN DEL TRAINING\n\n")

  return newGeneration, f_, bestThresold, predictionList

# TRAINING DATA

In [132]:
df_categories = pd.DataFrame({'Categories': df_clean['FirstLevel'].drop_duplicates(), 'Values': {}}).sort_values('Categories').reset_index(drop=True)
g = df_clean['FirstLevel'].value_counts()

i__ = 0
for key, value in g.items():
    mask = df_categories['Categories'] == key
    df_categories.loc[mask, 'Values'] = value
    i__+=1

df_categories

/tmp/ipykernel_793/3913633337.py:1: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df_categories = pd.DataFrame({'Categories': df_clean['FirstLevel'].drop_duplicates(), 'Values': {}}).sort_values('Categories').reset_index(drop=True)


,Categories,Values
0,A,35.0
1,B,48.0
2,C,121.0
3,D,311.0
4,E,52.0
5,F,296.0
6,G,106.0
7,H,377.0
8,I,226.0
9,J,86.0


In [133]:
def getTFByCategories(data,df_categories):
  dataFBRM = {}
  for index, row in df_categories.iterrows():
    k, t, r = algorithmFBRM(row,data[data['FirstLevel'].apply(lambda x: row['Categories'] in x)])
  
    tf_keywords = count_terms_frecuency(k)
    tf_title = count_terms_frecuency(t)
    tf_keywords_title = count_terms_frecuency(r)
    
    dataFBRM[row['Categories']] = {'tf_keywords': tf_keywords, 'tf_title': tf_title, 'tf_keywords_title': tf_keywords_title}

  #print(dataFBRM['B']['tf_keywords_title']['system']) # test
  return dataFBRM

In [134]:
log_Data = getTFByCategories(df_Processed_Data,df_categories)

In [135]:
print(log_Data['A']['tf_keywords']['knowledge'])
print(log_Data['A']['tf_title']['knowledge'])
print(log_Data['A']['tf_keywords_title']['knowledge'])


11
15
26


In [136]:
def getFeaturesMap(dataFBRM):

  ### Seleccionando caracteristicas
  featuresSelected = [] # get all words by Train
  for category, value in dataFBRM.items(): #loop by categories

    feature = list(value['tf_keywords_title'].keys()) # get words total by Category
    featuresSelected += feature

  featuresSelected = set(featuresSelected)
  featuresSelected = list(featuresSelected)
  featuresSelected.sort()

  ### nuevo dataframe
  # df_featuresMap = pd.DataFrame(columns=['Names'] + ['Metadata'] + featuresSelected)
  # # print(" features Map: ", df_featuresMap)
  # display(df_featuresMap)

  data_list = []
  
  for category, metadata in dataFBRM.items():
    for key, features in metadata.items():
      nameRow = {'Names': category, 'Metadata': key}
      nameRow.update(features)
      data_list.append(nameRow)
  
  df_featuresMap = pd.DataFrame(data_list)

  ### Preprocesando el dataframe de caracteristicas
  df_featuresMap = df_featuresMap.fillna(0)
  # display(df_featuresMap)
  
  df_features= df_featuresMap.iloc[:, 2:]
  df_description= df_featuresMap.iloc[:, :2]

  sumCol = df_features.sum()
  df_features = df_features[sumCol.sort_values(ascending=False).index]

  df_featuresMap = pd.concat([df_description, df_features], axis=1)
  display(df_featuresMap)
  return df_featuresMap

In [137]:
def getTFByDoc(train_data):

    dataFBRM_train_kt = []
    dataFBRM_train_k = []
    dataFBRM_train_t = []

    for i in range(len(train_data)):
      # print(train_data.iloc[i])
      train_df = pd.DataFrame([train_data.iloc[i]])
      k, t, r = algorithmFBRM(0,train_df)
      
      tf_keywords = count_terms_frecuency(k)
      tf_title = count_terms_frecuency(t)
      tf_keywords_title = count_terms_frecuency(r)
    
      dataFBRM_train_kt.append(tf_keywords_title)
      dataFBRM_train_k.append(tf_keywords)
      dataFBRM_train_t.append(tf_title)

    return dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt

In [138]:
nInd = 10
# nGenes = 56342
pCrossover = 0.5
pMutation = 0.9
pCrossover_BLX_Alpha = 0.5
threshold = 0.1
pMutation = 0.5
iteration = 50

In [139]:
%%time
k = 3
# kf = KFold(n_splits=k,)
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# train and test data split
train_list = []
test_list = []

oldDf_FeaturesMap = []
bestOffstring_list = []
bestThresold_list = []
bestPredictionList = []

f_list = []

for train_index, test_index in kf.split(df_Processed_Data):
    train_data, test_data = df_Processed_Data.iloc[train_index], df_Processed_Data.iloc[test_index]

    ## save data
    train_list.append(train_data)
    test_list.append(test_data)

    ## TF
    dataFBRM = getTFByCategories(train_data,df_categories)

    ## Create a Features Map
    df_featuresMap = getFeaturesMap(dataFBRM)
    oldDf_FeaturesMap.append(df_featuresMap)

    ### TF de cada articulo
    dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(train_data)
    
    # train_data, panda train
    # nInd, nro of sample
    # df_featuresMap, Panda features of words
    # dataFBRM_train_kt, List of object (word TF by Doc)
    # pCrossover, umbral of crossover
    # pMutation, umbral of mutation
    # iteration, nro of iteration
    # blend_crossover_blx_alpha, function of crossover
    # pCrossover_BLX_Alpha, value alpha
    # threshold,

    ## Entrenamiento con Algoritmo Genetic0
    bestOffstring, f_, bestThresold, predictionList = geneticAlgorithms(train_data,nInd,df_featuresMap,dataFBRM_train_kt,pCrossover,pMutation,iteration,blend_crossover_blx_alpha,pCrossover_BLX_Alpha,threshold)

    bestOffstring_list.append(bestOffstring)
    bestThresold_list.append(bestThresold)
    bestPredictionList.append(predictionList)
    f_list.append(f_)

,Names,Metadata,information,knowledge,systems,management,system,learning,state,formal,...,transclusions,fine-grained,provably,rule-based,gossip,correctness,scalable,cyclic,reference,game
0,A,tf_keywords,1.0,7.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,1.0,8.0,0.0,8.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,2.0,15.0,0.0,13.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,2.0,0.0,3.0,3.0,6.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,3.0,3.0,4.0,2.0,3.0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,5.0,3.0,7.0,5.0,9.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,4.0,1.0,8.0,1.0,6.0,0.0,8.0,8.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


OBTENER EL MEJOR THRESHOLD

 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1


the best Thresold:  0.1
the best Acurracy:  0.4546420460827386
Con threshold:  0.11
the best Thresold:  0.11
the best Acurracy:  0.4660599083734771
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
the best Thresold:  0.14
the best Acurracy:  0.4785885075077027
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
Con threshold:  0.17000000000000004
Con threshold:  0.18000000000000005
Con threshold:  0.19000000000000006
Con threshold:  0.20000000000000007
mejores resultados:  [0.4785885075077027] 	-> mejor thresold [0.14]

-------ITERACION  1  ----------


 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ---------------------------------

,Names,Metadata,knowledge,systems,management,information,system,design,formal,based,...,emerging,kgc,malicious,error-correcting,reuse,closed,worlds,secret,components,np-hardness
0,A,tf_keywords,9.0,0.0,6.0,4.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,13.0,0.0,13.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,22.0,0.0,19.0,5.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,1.0,2.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,0.0,3.0,0.0,1.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,1.0,2.0,1.0,0.0,3.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,1.0,5.0,1.0,1.0,5.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,2.0,9.0,0.0,2.0,5.0,7.0,8.0,9.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


OBTENER EL MEJOR THRESHOLD

 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.4686733844751172
Con threshold:  0.11
Con threshold:  0.12
the best Thresold:  0.12
the best Acurracy:  0.4737643924215989
Con threshold:  0.13
Con threshold:  0.14
the best Thresold:  0.14
the best Acurracy:  0.4801027089133137
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
Con threshold:  0.17000000000000004
Con threshold:  0.18000000000000005
Con threshold:  0.19000000000000006
Con threshold:  0.20000000000000007
mejores resultados:  [0.4801027089133137] 	-> mejor thresold [0.14]

-------ITERACION  1  ----------


 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ---------------------------

,Names,Metadata,knowledge,systems,management,information,system,learning,formal,software,...,smells,intuitionistic,immersive,bipartite,-stable,conversation,closed,string,worlds,game
0,A,tf_keywords,6.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,9.0,0.0,7.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,15.0,0.0,10.0,3.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,0.0,2.0,3.0,1.0,4.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,2.0,4.0,3.0,3.0,2.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,2.0,6.0,6.0,4.0,6.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,3.0,9.0,1.0,4.0,5.0,2.0,6.0,13.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


OBTENER EL MEJOR THRESHOLD

 ------------------------------------ POBLACION:  1  ------------------------------------ 

Con threshold:  0.1
the best Thresold:  0.1
the best Acurracy:  0.49489857371190926
Con threshold:  0.11
Con threshold:  0.12
Con threshold:  0.13
Con threshold:  0.14
Con threshold:  0.15000000000000002
Con threshold:  0.16000000000000003
mejores resultados:  [0.49489857371190926] 	-> mejor thresold [0.1]

-------ITERACION  1  ----------


 ------------------------------------ POBLACION:  1  ------------------------------------ 


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  -------------------

In [165]:
print(len(train_list))
print(len(test_list))

3
3


In [166]:
train_list[0]

,ID,Title,Keywords,FirstLevel,Processed_Title,Processed_Keyword,Processed_T_K
0,1,Integration of Communities into Process-Orient...,"cooperative knowledge generation,knowledge co...",['H'],process-oriented structures,cooperative knowledge generation,process-oriented structures cooperative knowle...
1,3,Small Groups Learning Synchronously Online at ...,"professional training,workplace learning,compu...",['H' 'J'],small groups learning synchronously online,computer-supported cooperative learning,small groups learning synchronously online com...
4,6,Tube Map Visualization: Evaluation of a Novel ...,"knowledge visualization,information visualiza...",['H'],tube map visualization,knowledge visualization,tube map visualization knowledge visualization
7,9,"KMDL - Capturing, Analysing and Improving Know...","Process-oriented Knowledge Management,knowledg...",['D' 'H' 'I'],improving knowledge-intensive business processes,knowledge modeling description language,improving knowledge-intensive business process...
8,10,The Role of Knowledge Management Solutions in ...,"knowledge management,business process,enterpri...",['A' 'H'],knowledge management solutions,knowledge management,knowledge management solutions knowledge manag...
...,...,...,...,...,...,...,...
1093,1469,Innovation and Quality in e-Learning: a Europe...,"innovation,quality,e-learning territories,visi...",['K'],european perspective,e-learning territories,european perspective e-learning territories
1094,1470,A Meta-modeling based Approach for the Multi-D...,"model-driven development,multi-disciplinary de...",['D' 'K'],meta-modeling based approach,model-driven development,meta-modeling based approach model-driven deve...
1095,1471,Application Scenarios for the Learning Objects...,"Learning Object,Learning Objects Repository,e-...",['H' 'K'],learning objects pool,learning objects repository,learning objects pool learning objects repository
1096,1472,TQ-Bot: An AIML-based Tutor and Evaluator Bot,"AIML,ALICE,BDI,chatterbots,Claroline,Intellige...",['I' 'K'],aiml-based tutor,intelligent tutoring systems,aiml-based tutor intelligent tutoring systems


In [167]:
test_list[0]

,ID,Title,Keywords,FirstLevel,Processed_Title,Processed_Keyword,Processed_T_K
2,4,Using Weblogs for Knowledge Sharing and Learni...,"Experience-based Information System,wiki,weblo...",['A' 'D' 'H' 'J' 'K'],knowledge sharing,micro-didactical learning arrangement,knowledge sharing micro-didactical learning ar...
3,5,Modelling and Implementing Pre-built Informati...,"modelling method,introduction method,context-a...",['H' 'I' 'J'],implementing pre-built information spaces,modelling method,implementing pre-built information spaces mode...
5,7,Reconciling Knowledge Management and Workflow ...,"workflow,knowledge management",['H'],activity-based knowledge management approach,knowledge management,activity-based knowledge management approach k...
6,8,A Methodology and a Toolkit that Integrate Tec...,"knowledge management,knowledge networks,inter-...",['C' 'I'],integrate technological,inter-organizational networked businesses,integrate technological inter-organizational n...
10,13,A Knowledge Infrastructure Hierarchy Model for...,"knowledge management,information systems,data ...",['H'],knowledge infrastructure hierarchy model,knowledge management,knowledge infrastructure hierarchy model knowl...
...,...,...,...,...,...,...,...
1091,1466,Effective Computability of Solutions of Differ...,"ordinary differential equations,differential i...",['F' 'G'],ten thousand monkeys approach,ordinary differential equations,ten thousand monkeys approach ordinary differe...
1097,1473,Eduquito: Virtual Environment for Digital Incl...,"virtual environment,accessibility,persons with...",['K'],virtual environment,virtual environment,virtual environment virtual environment
1099,1475,Semantic Spiral Timelines Used as Support for ...,"visualization,e-learning,timeline,spiral,Moodle",['L' 'M'],semantic spiral timelines,visualization,semantic spiral timelines visualization
1100,1476,Visualization of Syntax Trees for Language Pro...,"syntax trees,visualization,educational software",['D' 'K'],language processing courses,syntax trees,language processing courses syntax trees


In [154]:
oldDf_FeaturesMap[0]

,Names,Metadata,information,knowledge,systems,management,system,learning,state,formal,...,transclusions,fine-grained,provably,rule-based,gossip,correctness,scalable,cyclic,reference,game
0,A,tf_keywords,1.0,7.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,1.0,8.0,0.0,8.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,2.0,15.0,0.0,13.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,2.0,0.0,3.0,3.0,6.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,3.0,3.0,4.0,2.0,3.0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,5.0,3.0,7.0,5.0,9.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,4.0,1.0,8.0,1.0,6.0,0.0,8.0,8.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0


In [168]:
bestOffstring_list[0]

array([[ 1.73975537,  9.03893623,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.75061224, 10.58772962,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.93736223,  9.51643487,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.73546047,  9.72659027,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.73975537,  9.03893623,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.59626806,  9.49274442,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [169]:
bestThresold_list

[0.14, 0.14, 0.1]

In [170]:
f_list

[[0.4819515829058351,
  0.4791865596695628,
  0.48427738260999964,
  0.4792606005567552,
  0.4800268127586172,
  0.47944954896530734,
  0.473075736547044,
  0.4703946546418184,
  0.48257067237505025,
  0.48109761468367956],
 [0.48797046020672663,
  0.48987349249912043,
  0.492653177974852,
  0.47469565245131556,
  0.4916649605917974,
  0.49508117151202713,
  0.493190729150412,
  0.4916649605917974,
  0.49508117151202713,
  0.49508117151202713],
 [0.49035455778332265,
  0.49534951928268367,
  0.49499632160834783,
  0.4957763390239806,
  0.493840263896704,
  0.49684267146629124,
  0.49716948003113665,
  0.49825953877095336,
  0.49825953877095336,
  0.49825953877095336]]

In [147]:
# f_list = []
# for i in range(len(bestOffstring_list)):

#   dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(train_list[i])
#   exp_list = train_list[i]['FirstLevel'].tolist()
#   datadf_chromosome = oldDf_FeaturesMap[i][oldDf_FeaturesMap[i]['Metadata'] != 'tf_keywords_title']
#   # print(datadf_chromosome)
#   df_features = datadf_chromosome.iloc[:, 2:]

#   f_ = getFittness(bestOffstring_list[i],oldDf_FeaturesMap[i],df_features.shape[1],nInd,bestOffstring_list[i].shape[0] * bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
#   f_list.append(f_)


## TESTING F1 SCORE

In [197]:
%%time
k = 3
f_test_list = []
p_test_bestPredictionList = []
e_test_exp_list = []

for i in range(len(bestOffstring_list)):

  dataFBRM_test_k, dataFBRM_test_t, dataFBRM_test_kt = getTFByDoc(test_list[i])
  exp_list = test_list[i]['FirstLevel'].tolist()

  ## TF
  dataFBRM_i= getTFByCategories(train_list[i],df_categories)

  ## Create a Features Map
  df_featuresMap_i = getFeaturesMap(dataFBRM_i)

  ## hallar el df_featuresMap
  datadf_chromosome = df_featuresMap_i[df_featuresMap_i['Metadata'] != 'tf_keywords_title']

  df_features = datadf_chromosome.iloc[:, 2:]
  # display(df_features)

  f_, predictionList = getFittnessToAccuracy(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_test_kt,exp_list,bestThresold_list[i])
  # f_, thresholdByChromosome, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
  # f_, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,0.1)

  f_test_list.append(f_)
  p_test_bestPredictionList.append(predictionList)
  e_test_exp_list.append(exp_list)



,Names,Metadata,information,knowledge,systems,management,system,learning,state,formal,...,transclusions,fine-grained,provably,rule-based,gossip,correctness,scalable,cyclic,reference,game
0,A,tf_keywords,1.0,7.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,1.0,8.0,0.0,8.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,2.0,15.0,0.0,13.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,2.0,0.0,3.0,3.0,6.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,3.0,3.0,4.0,2.0,3.0,0.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,5.0,3.0,7.0,5.0,9.0,0.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,4.0,1.0,8.0,1.0,6.0,0.0,8.0,8.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 



/tmp/ipykernel_793/604264698.py:64: RuntimeWarning: divide by zero encountered in divide
  uD = w_title / weightSum
/tmp/ipykernel_793/604264698.py:64: RuntimeWarning: invalid value encountered in divide
  uD = w_title / weightSum
/tmp/ipykernel_793/604264698.py:60: RuntimeWarning: divide by zero encountered in divide
  uD = w_keywords / weightSum
/tmp/ipykernel_793/604264698.py:60: RuntimeWarning: invalid value encountered in divide
  uD = w_keywords / weightSum
/tmp/ipykernel_793/604264698.py:68: RuntimeWarning: divide by zero encountered in divide
  uD = w_keywords_title / weightSum
/tmp/ipykernel_793/604264698.py:68: RuntimeWarning: invalid value encountered in divide
  uD = w_keywords_title / weightSum



 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ------------------------------------ 


 ------------------------------------ POBLACION:  7  ------------------------------------ 


 ------------------------------------ POBLACION:  8  ------------------------------------ 


 ------------------------------------ POBLACION:  9  ------------------------------------ 


 ------------------------------------ POBLACION:  10  ------------------------------------ 



,Names,Metadata,knowledge,systems,management,information,system,design,formal,based,...,emerging,kgc,malicious,error-correcting,reuse,closed,worlds,secret,components,np-hardness
0,A,tf_keywords,9.0,0.0,6.0,4.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,13.0,0.0,13.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,22.0,0.0,19.0,5.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,1.0,2.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,0.0,3.0,0.0,1.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,1.0,2.0,1.0,0.0,3.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,1.0,5.0,1.0,1.0,5.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,2.0,9.0,0.0,2.0,5.0,7.0,8.0,9.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0


...


 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ------------------------------------ 


 ------------------------------------ POBLACION:  7  ------------------------------------ 


 ------------------------------------ POBLACION:  8  ------------------------------------ 


 ------------------------------------ POBLACION:  9  ------------------------------------ 


 ------------------------------------ POBLACION:  10  ------------------------------------ 



,Names,Metadata,knowledge,systems,management,information,system,learning,formal,software,...,smells,intuitionistic,immersive,bipartite,-stable,conversation,closed,string,worlds,game
0,A,tf_keywords,6.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A,tf_title,9.0,0.0,7.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A,tf_keywords_title,15.0,0.0,10.0,3.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,B,tf_keywords,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,B,tf_title,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,B,tf_keywords_title,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C,tf_keywords,0.0,2.0,3.0,1.0,4.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,C,tf_title,2.0,4.0,3.0,3.0,2.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,C,tf_keywords_title,2.0,6.0,6.0,4.0,6.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,D,tf_keywords,3.0,9.0,1.0,4.0,5.0,2.0,6.0,13.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0



 ------------------------------------ POBLACION:  1  ------------------------------------ 



/tmp/ipykernel_793/604264698.py:64: RuntimeWarning: divide by zero encountered in divide
  uD = w_title / weightSum
/tmp/ipykernel_793/604264698.py:64: RuntimeWarning: invalid value encountered in divide
  uD = w_title / weightSum
/tmp/ipykernel_793/604264698.py:60: RuntimeWarning: divide by zero encountered in divide
  uD = w_keywords / weightSum
/tmp/ipykernel_793/604264698.py:60: RuntimeWarning: invalid value encountered in divide
  uD = w_keywords / weightSum
/tmp/ipykernel_793/604264698.py:68: RuntimeWarning: divide by zero encountered in divide
  uD = w_keywords_title / weightSum
/tmp/ipykernel_793/604264698.py:68: RuntimeWarning: invalid value encountered in divide
  uD = w_keywords_title / weightSum



 ------------------------------------ POBLACION:  2  ------------------------------------ 


 ------------------------------------ POBLACION:  3  ------------------------------------ 


 ------------------------------------ POBLACION:  4  ------------------------------------ 


 ------------------------------------ POBLACION:  5  ------------------------------------ 


 ------------------------------------ POBLACION:  6  ------------------------------------ 


 ------------------------------------ POBLACION:  7  ------------------------------------ 


 ------------------------------------ POBLACION:  8  ------------------------------------ 


 ------------------------------------ POBLACION:  9  ------------------------------------ 


 ------------------------------------ POBLACION:  10  ------------------------------------ 

CPU times: user 29.5 s, sys: 70 ms, total: 29.6 s
Wall time: 29.5 s


In [198]:
f_list

[[0.4819515829058351,
  0.4791865596695628,
  0.48427738260999964,
  0.4792606005567552,
  0.4800268127586172,
  0.47944954896530734,
  0.473075736547044,
  0.4703946546418184,
  0.48257067237505025,
  0.48109761468367956],
 [0.48797046020672663,
  0.48987349249912043,
  0.492653177974852,
  0.47469565245131556,
  0.4916649605917974,
  0.49508117151202713,
  0.493190729150412,
  0.4916649605917974,
  0.49508117151202713,
  0.49508117151202713],
 [0.49035455778332265,
  0.49534951928268367,
  0.49499632160834783,
  0.4957763390239806,
  0.493840263896704,
  0.49684267146629124,
  0.49716948003113665,
  0.49825953877095336,
  0.49825953877095336,
  0.49825953877095336]]

In [199]:
f_test_list

[[0.6795076726342711,
  0.6788682864450127,
  0.6780690537084398,
  0.6782289002557543,
  0.6793478260869563,
  0.6799872122762148,
  0.6801470588235293,
  0.6809462915601021,
  0.6803069053708439,
  0.6807864450127877],
 [0.6797563712133355,
  0.6787946786343966,
  0.6779932681519476,
  0.6775124218624781,
  0.6786343965379066,
  0.6784741144414168,
  0.6789549607308865,
  0.6786343965379066,
  0.6784741144414168,
  0.6784741144414168],
 [0.6300689213014906,
  0.6335951274242667,
  0.6307100496874498,
  0.6321525885558583,
  0.6321525885558583,
  0.6323128706523482,
  0.6316717422663888,
  0.6316717422663888,
  0.6316717422663888,
  0.6316717422663888]]

# Busqueda del nuevo threshold en los offtrings

In [178]:
idxBestResultTrain = []
bestThresoldNew = []
bestOffstringIdx = []

for i in range(len(f_list)):
  maxV = 0
  idxMax = 0
  for j in range(len(f_list[i])):
    if(f_list[i][j] > maxV):
      maxV = f_list[i][j]
      idxMax = j

  ## TF by Doc
  dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(train_list[i])
  exp_list = train_list[i]['FirstLevel'].tolist()

  ## TF
  dataFBRM_i = getTFByCategories(train_list[i])

  ## Create a Features Map
  df_featuresMap_i = getFeaturesMap(dataFBRM_i)

  ## hallar el df_featuresMap
  datadf_chromosome = df_featuresMap_i[df_featuresMap_i['Metadata'] != 'tf_keywords_title']

  df_features = datadf_chromosome.iloc[:, 2:]
  # display(df_features)


  candidateGeneration = [bestOffstring_list[i][idxMax]]
  fitnessByChromosome, thresholdByChromosome = getThresold(candidateGeneration,df_featuresMap,df_features.shape[1],nInd,len(candidateGeneration),dataFBRM_train_kt,exp_list,threshold)
  print("mejores resultados: ",fitnessByChromosome, "\t-> mejor thresold",thresholdByChromosome)
  idx_max= fitnessByChromosome.index(max(fitnessByChromosome))
  bestThresold = thresholdByChromosome[idx_max]
  bestThresoldNew.append(bestThresold)
  bestOffstringIdx.append(idxMax)

TypeError: getTFByCategories() missing 1 required positional argument: 'df_categories'

# Seleecionando los mejores resultados

In [177]:
idxBestResultTest

NameError: name 'idxBestResultTest' is not defined

# ACCURACY

In [ ]:
medianAcurracy = 0
for i in range(len(idxBestResultTest)):
  medianAcurracy += f_test_list[i][idxBestResultTest[i]]
print(medianAcurracy/len(idxBestResultTest))

In [ ]:
print(len(p_test_bestPredictionList[0]))
print(len(e_test_exp_list[0]))


In [ ]:
for i in range(len(idxBestResultTest)):
  predList = p_test_bestPredictionList[i][idxBestResultTest[i]]
  expList = e_test_exp_list[i]
  # print(i)
  # print(len(expList))
  print("\nTHE BEST RESULT\n")
  for j in range(len(predList)):
    print("y_exp: ",expList[j] , "\t->", predList[j])


# Extrayendo el Mapa de TF

In [ ]:
# aca solo se saca el olds para ver la estructura
display(oldDf_FeaturesMap[0])
df_newFM = oldDf_FeaturesMap[0].copy()

population = bestOffstring_list[0][idxBestResultTest[0]]

# ## hallar el df_featuresMap
# datadf_chromosome = df_newFM[df_newFM['Metadata'] != 'tf_keywords_title']

df_features = df_newFM.iloc[:, 2:]
df_categories_metadata = df_newFM.iloc[:, :2]
# print(df_features.shape[0])

display(df_features)
display(df_categories_metadata)


# columns = list(df_featuresMap.iloc[:, 2:].columns)
nRow = len(population) // df_features.shape[1]

blocks = np.split(population, nRow)
print(blocks[0])

i = 0
k = 0
while(i < 39):
  for j in range(len(blocks[k])):
    df_features.iloc[i, j] = blocks[k][j]
  k+=2
  i+=3

i = 1
k = 1
while(i < 39):
  for j in range(len(blocks[k])):
    df_features.iloc[i, j] = blocks[k][j]
  k+=2
  i+=3

i = 2
k = 0
while(i < 39):
  for j in range(len(blocks[k])):
    df_features.iloc[i, j] = blocks[k][j] + blocks[k+1][j]
  k+=2
  i+=3



In [ ]:
df_OffString = pd.concat([df_categories_metadata, df_features], axis=1)
df_OffString

## TESTING ACCURACY

In [ ]:
%%time
k = 3
f_test_list = []
p_test_bestPredictionList = []
e_test_exp_list = []

for i in range(len(bestOffstring_list)):

  dataFBRM_train_k, dataFBRM_train_t, dataFBRM_train_kt = getTFByDoc(test_list[i])
  exp_list = test_list[i]['FirstLevel'].tolist()

  ## TF
  dataFBRM_i = getTFByCategories(train_list[i])

  ## Create a Features Map
  df_featuresMap_i = getFeaturesMap(dataFBRM_i)

  ## hallar el df_featuresMap
  datadf_chromosome = df_featuresMap_i[df_featuresMap_i['Metadata'] != 'tf_keywords_title']

  df_features = datadf_chromosome.iloc[:, 2:]
  # display(df_features)

  f_, predictionList = getFittnessToAccuracy(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
  # f_, thresholdByChromosome, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,bestThresold_list[i])
  # f_, predictionList = getFittness(bestOffstring_list[i],df_featuresMap_i,df_features.shape[1],nInd,bestOffstring_list[i].shape[1],dataFBRM_train_kt,exp_list,0.1)

  f_test_list.append(f_)
  p_test_bestPredictionList.append(predictionList)
  e_test_exp_list.append(exp_list)



In [ ]:
f_list

In [ ]:
f_test_list

# Seleccionando los mejores resultados

In [ ]:
idxBestResultTest = []

for i in range(len(f_test_list)):
  maxV = 0
  idxMax = 0
  for j in range(len(f_test_list[i])):
    if (f_test_list[i][j] > maxV):
      maxV = f_test_list[i][j]
      idxMax = j
  idxBestResultTest.append(idxMax)

In [ ]:
idxBestResultTest

# ACCURACY

In [ ]:
medianAcurracy = 0
for i in range(len(idxBestResultTest)):
  medianAcurracy += f_test_list[i][idxBestResultTest[i]]
print(medianAcurracy/len(idxBestResultTest))

In [ ]:
print(len(p_test_bestPredictionList[0]))
print(len(e_test_exp_list[0]))


In [ ]:
for i in range(len(idxBestResultTest)):
  predList = p_test_bestPredictionList[i][idxBestResultTest[i]]
  expList = e_test_exp_list[i]
  # print(i)
  # print(len(expList))
  print("\nTHE BEST RESULT\n")
  for j in range(len(predList)):
    print("y_exp: ",expList[j] , "\t\t y_pred ->", predList[j])

# Interpretabilidad

In [ ]:
def getTfByDoc(test_list):
    k = []
    t = []
    r = []
    t_text = test_list[0].iloc[2]['Title_clean'] ## 1 k fold, fila 0 y columna title
    k_text = test_list[0].iloc[2]['Keywords_clean'] ## 1 k fold, fila 0 y columna keywords
    y_exp = test_list[0].iloc[2]['FirstLevel'] ## 1 k fold, fila 0 y columna keywords

    print("Etiquetas esperadas: ",y_exp)
    # print(t_text)
    # print(k_text)

    ## algorithmFBRM
    titles = re.split(r'##', t_text)
    keywords = re.split(r'##', k_text)

    for text in titles:
      k.append(text)
    for text in keywords:
      t.append(text)
    r = k + t

    tf_keywords = count_terms_frecuency(k)
    tf_title = count_terms_frecuency(t)
    tf_keywords_title = count_terms_frecuency(r)

    # print(tf_keywords)
    # print(tf_title)
    # print(tf_keywords_title)
    return tf_keywords, tf_title, tf_keywords_title


In [ ]:
tf_keywords, tf_title, tf_keywords_title = getTfByDoc(test_list)
tf_keywords_title

In [ ]:
# aca solo se saca el olds para ver la estructura
# display(oldDf_FeaturesMap[0])
display(df_OffString)
df_newFM = oldDf_FeaturesMap[0].copy()
name_columns = list(df_newFM.iloc[:,2:].columns)
# print(name_columns)

population = bestOffstring_list[0][idxBestResultTest[0]]

# ## hallar el df_featuresMap
# datadf_chromosome = df_newFM[df_newFM['Metadata'] != 'tf_keywords_title']

df_features = df_newFM.iloc[:, 2:]
df_categories_metadata = df_newFM.iloc[:, :2]
# print(df_features.shape[0])

# display(df_features)
# display(df_categories_metadata)


# columns = list(df_featuresMap.iloc[:, 2:].columns)
nRow = len(population) // df_features.shape[1]

blocks = np.split(population, nRow)
# print(blocks[0])

## data de prueba
tf_keywords, tf_title, tf_keywords_title = getTfByDoc(test_list)

# i = 0
# k = 0
# while(i < 39):
#   for j in range(len(blocks[k])):
#     df_features.iloc[i, j] = blocks[k][j]
#   k+=2
#   i+=3

# i = 1
# k = 1
# while(i < 39):
#   for j in range(len(blocks[k])):
#     df_features.iloc[i, j] = blocks[k][j]
#   k+=2
#   i+=3


idxFindOfDoc = [] # indices de la population
valueOfDict = [] # valores (pesos) del testing

for key, value in tf_keywords_title.items():
    if key in name_columns:
        idxFindOfDoc.append(name_columns.index(key))
        valueOfDict.append(value)
print(idxFindOfDoc)

i = 2
k = 0
c_cont = 0
sum_total = []
while(i < 39):
    print("Categoria por titulo + keywords: ", df_categories.iloc[c_cont]['Categories'])

    c_cont += 1
    sum_ = 0
    v_idx = 0
    for idx in idxFindOfDoc:
        weigth_kt = blocks[k][idx] + blocks[k+1][idx]
        v_ = valueOfDict[v_idx]
        weigthxvalue = weigth_kt * v_
        sum_ += weigthxvalue
        v_idx +=1

        print(name_columns[idx] ,"\t:", v_," x ",weigth_kt, " + ")
    print(" = ",sum_)
    sum_total.append(sum_)

    k+=2
    i+=3

v_sum = sum(sum_total)
print("Suma total: ", v_sum)
for i in range(len(sum_total)):
    print("Categoria por titulo + keywords: ", df_categories.iloc[i]['Categories'], "es: ", sum_total[i]/v_sum)


In [ ]:

dictToWordClouds = {}

for clave, valor in zip(df_categories['Categories'], sum_total):
    dictToWordClouds[clave] = valor

print(dictToWordClouds)


In [ ]:
max_frequency = max(dictToWordClouds.values())
scaling_factor = 3
scaled_frequencies = {word: scaling_factor * (frequency / max_frequency) for word, frequency in dictToWordClouds.items()}

text = ' '.join([word for word in scaled_frequencies for _ in range(round(scaled_frequencies[word]))])
# print(text)
nube_palabras = WordCloud(width=800, height=800, background_color='white').generate(text)

plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(nube_palabras)
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()